In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [32]:
data = pd.read_csv("https://github.com/YBIFoundation/Dataset/raw/main/Cancer.csv")

In [34]:
class BayesianDecisionTheory:
    def __init__(self, priors, likelihoods, loss_matrix=None):
        """
        Initialize the Bayesian Decision Theory class.

        :param priors: Array of prior probabilities P(Ci)
        :param likelihoods: 2D array of likelihood probabilities P(X|Ci)
                            Rows correspond to classes, columns to features.
        :param loss_matrix: 2D array of loss values λik (optional).
                            Defaults to 0/1 loss.
        """
        self.priors = np.array(priors)
        self.likelihoods = np.array(likelihoods)
        self.num_classes = len(priors)
        self.loss_matrix = loss_matrix if loss_matrix is not None else self._default_loss_matrix()

    def _default_loss_matrix(self):
        """
        Generate a default 0/1 loss matrix.
        """
        loss = np.ones((self.num_classes, self.num_classes))
        np.fill_diagonal(loss, 0)
        return loss

    def posterior_probabilities(self, feature_vector):
        """
        Calculate posterior probabilities for a given feature vector.

        :param feature_vector: Array of observed features X.
        :return: Array of posterior probabilities P(Ci|X).
        """
        evidence = sum(
            self.priors[i] * np.prod(self.likelihoods[i] ** feature_vector)
            for i in range(self.num_classes)
        )
        posteriors = [
            (self.priors[i] * np.prod(self.likelihoods[i] ** feature_vector)) / evidence
            for i in range(self.num_classes)
        ]
        return np.array(posteriors)

    def classify(self, feature_vector):
        """
        Classify a sample based on the posterior probabilities.

        :param feature_vector: Array of observed features X.
        :return: Predicted class index.
        """
        posteriors = self.posterior_probabilities(feature_vector)
        return np.argmax(posteriors)

    def calculate_risk(self, feature_vector):
        """
        Calculate the risk for each class.

        :param feature_vector: Array of observed features X.
        :return: Array of risks for each class.
        """
        posteriors = self.posterior_probabilities(feature_vector)
        risks = [np.dot(self.loss_matrix[i], posteriors) for i in range(self.num_classes)]
        return np.array(risks)

    def classify_with_risk(self, feature_vector):
        """
        Classify a sample based on the risk minimization principle.

        :param feature_vector: Array of observed features X.
        :return: Predicted class index based on minimum risk.
        """
        risks = self.calculate_risk(feature_vector)
        return np.argmin(risks)

    def classify_with_reject(self, feature_vector, threshold=0.5):
        """
        Classify a sample with a reject option based on a posterior threshold.

        :param feature_vector: Array of observed features X.
        :param threshold: Posterior probability threshold for rejection.
        :return: Predicted class index or -1 for rejection.
        """
        posteriors = self.posterior_probabilities(feature_vector)
        max_posterior = np.max(posteriors)
        if max_posterior < threshold:
            return -1  # Reject class
        return np.argmax(posteriors)



In [5]:
# Example Usage

# Define priors, likelihoods, and a custom loss matrix
priors = [0.4, 0.6]  # Prior probabilities for two classes
likelihoods = [
    [0.2, 0.8],  # Likelihoods for class C1
    [0.6, 0.4]   # Likelihoods for class C2
]
loss_matrix = [
    [0, 2],  # Loss for predicting C1 when true class is C1 or C2
    [1, 0]   # Loss for predicting C2 when true class is C1 or C2
]
feature_vector = [1, 0]  # Observed feature vector

# Create the Bayesian Decision Theory model
model = BayesianDecisionTheory(priors, likelihoods, loss_matrix)

# Perform classifications
print("Posterior Probabilities:", model.posterior_probabilities(feature_vector))
print("Classified as:", model.classify(feature_vector))
print("Risk for each class:", model.calculate_risk(feature_vector))
print("Classified based on risk:", model.classify_with_risk(feature_vector))
print("Classified with rejection (threshold 0.5):", model.classify_with_reject(feature_vector, threshold=0.5))


Posterior Probabilities: [0.18181818 0.81818182]
Classified as: 1
Risk for each class: [1.63636364 0.18181818]
Classified based on risk: 1
Classified with rejection (threshold 0.5): 1


In [ ]:
url = "https://github.com/YBIFoundation/Dataset/raw/main/Cancer.csv"
try:
    dataset = pd.read_csv(url)
    print("Dataset downloaded successfully!")
    print(dataset.head())  # Display the first 5 rows
except Exception as e:
    print(f"An error occurred: {e}")

In [39]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [40]:
print(data.shape)

(569, 33)


In [41]:


data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})
X = data.iloc[:, 2:].values  
y = data['diagnosis'].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estimate priors and likelihoods
priors = [np.mean(y_train == 0), np.mean(y_train == 1)]
likelihoods = []

for class_idx in [0, 1]:
    class_features = X_train[y_train == class_idx]
    feature_means = np.mean(class_features, axis=0)
    feature_variances = np.var(class_features, axis=0)
    likelihoods.append([1 / np.sqrt(2 * np.pi * var) for var in feature_variances])

likelihoods = np.array(likelihoods)

# Define a custom loss matrix
loss_matrix = [
    [0, 2],  # Loss for predicting benign when true class is malignant
    [1, 0]   # Loss for predicting malignant when true class is benign
]

# Create the Bayesian Decision Theory model
model = BayesianDecisionTheory(priors, likelihoods, loss_matrix)

# Perform predictions and evaluate
predictions = [model.classify_with_risk(x) for x in X_test]

# Evaluate performance
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))
print("Classification Report:\n", classification_report(y_test, predictions))


/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Confusion Matrix:
 [[71  0]
 [43  0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        71
           1       0.00      0.00      0.00        43

    accuracy                           0.62       114
   macro avg       0.31      0.50      0.38       114
weighted avg       0.39      0.62      0.48       114



/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kaleb/projects/ML/Bayesian-decision-theory-implementation/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. U